In [ ]:
import uuid
import random
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from extras import helpers as helpers, create_table_leanx as ctl

from masterdata import values_Meritor as values, master_data, text_data
from processes import inventory_doc_data

In [ ]:
# text tables
for method in (
    text_data.domain_fixed_values, 
    text_data.sales_doc_types, 
    text_data.sales_organizations,
    text_data.distribution,
    text_data.sales_doc_item_categories,
    text_data.sales_doc_rejection_reasons,
    text_data.system_status,
    text_data.blocking_reasons,
    text_data.releases,
    text_data.purchasing_doc_types,
    text_data.purchasing_organizations
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        # all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        # all_cols_req = ctl.clean_columns(all_cols,table_name)
        all_cols_req = pd.DataFrame(columns= ctl.required_table_columns[table_name])
        df = pd.concat([all_cols_req, pd.DataFrame(v.values())])
        df.to_csv(f'data/inv/ocpm/text/{table_name}.csv', index=False)

In [ ]:
# master tables
for method in (
    master_data.users,
    master_data.customers_and_vendors, 
    master_data.plants, 
    master_data.materials, 
    master_data.material_support,
    master_data.routes,
    master_data.company_codes,
    master_data.currencyexchanges
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        # all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        # all_cols_req = ctl.clean_columns(all_cols,table_name)
        all_cols_req = pd.DataFrame(columns= ctl.required_table_columns[table_name])
        df = pd.concat([all_cols_req, pd.DataFrame(v.values())])
        df.to_csv(f'data/inv/ocpm/master/{table_name}.csv', index=False)

In [ ]:
def get_user_name(automation_probability: float):
    if random.random() <= automation_probability:
        return 'BATCH_JOB'
    return random.choice(list(values.om_users.keys()))

def get_time_consumption(start_date, planned_target_date, latest_date):
    return (latest_date - start_date) / (planned_target_date - start_date)

In [ ]:
MARC = pd.read_csv('data/inv/ocpm/master/MARC.csv')
MARA = pd.read_csv('data/inv/ocpm/master/MARA.csv')
MAKT = pd.read_csv('data/inv/ocpm/master/MAKT.csv')
LFB1 = pd.read_csv('data/inv/ocpm/master/LFB1.csv')
LFA1 = pd.read_csv('data/inv/ocpm/master/LFA1.csv')
KNB1 = pd.read_csv('data/inv/ocpm/master/KNB1.csv')
KNA1 = pd.read_csv('data/inv/ocpm/master/KNA1.csv')
MBEW = pd.read_csv('data/inv/ocpm/master/MBEW.csv')
MBEWH = pd.read_csv('data/inv/ocpm/master/MBEW.csv')

all_prices = {}
all_has_contract_probabilities = {}
all_matnr_names = {}
all_availabilities = {}

# get price and material group by MATNR
for nr in MARA['MATNR']:
    name = MAKT[MAKT['MATNR'] == nr]['MAKTX'].values[0]
    for k, v in values.om_material_groups.items():
        for mgrp, attr in v.items():
            for mat, details in attr['materials'].items():
                if name == mat:
                    all_prices[nr] = details['price']
                    all_matnr_names[nr] = mat
                    all_has_contract_probabilities[nr] = attr['has_contract_probability']
                    all_availabilities[nr] = attr['availability']

print(all_prices['MAT000'])

In [ ]:
def get_params():
    # basic informaiton
    company_code = random.choice(list(values.om_company_codes.keys()))
    plant = random.choice(values.om_company_codes[company_code]['plants'])
    konnr = str(uuid.uuid4())[-15:]
    kunnr = random.choice(list(KNB1[KNB1['BUKRS'] == company_code]['KUNNR']))
    lifnr = random.choice(list(LFB1[LFB1['BUKRS'] == company_code]['LIFNR']))
    vendor_name = LFA1[LFA1['LIFNR'] == lifnr]['NAME1'].values[0]
    purchasing_org=random.choice(values.om_plants[plant]['purchasing_orgs'])
    new_lifnr = random.choice(list(LFB1[LFB1['BUKRS'] == company_code]['LIFNR']))
    payment_term = random.choice(list(LFB1[LFB1['LIFNR'] == lifnr]['ZTERM']))
    requested_by = get_user_name(0.1)

    sales_org=random.choice(values.om_plants[plant]['purchasing_orgs'])
    sales_office=random.choice(list(values.om_sales_orgs[sales_org]['sales_offices'].keys()))
    doc_type_number = random.random()
    # sales_doc_type= 'ZDIR' if random.random() < 0.7 else random.choice(list(values.om_sales_doc_types.keys()))
    sales_doc_type = ('ZDIR' if (doc_type_number < 0.4) else
                      'ZOR' if (doc_type_number < 0.8) else 
                      random.choice(list(values.om_sales_doc_types.keys()))
                      ) 
    mapping_salesdoc = {'ZOR': '40', 'ZDLR': '20', 'ZDIR': '10', 'ZDIS': '30','ZEXP': '50'}
    distribution_channel=mapping_salesdoc[sales_doc_type]
    automation_rate = values.om_sales_orgs[sales_org]['Automation_rate'][distribution_channel]

    #MATERIALS
    all_matnrs = MARC[MARC['WERKS'] == plant]['MATNR'].unique()
    #OM Materials 
    om_matnrs = random.sample(['MAT400','MAT401','MAT402','MAT403','MAT404'], 1)
    # Production BOM materials related
    bom_matnrs = [k for k, v in values.om_material_bom[om_matnrs[0]]["materials"].items()]
    #Proc materials
    matnrs = random.sample(list(all_matnrs), min(random.randint(5, 25), len(all_matnrs)))

    # QUANTITIES
    # OM quantities
    om_quantities = [random.randint(1, 4) for m in range(len(om_matnrs))]
    # Prod Bom quantities
    bom_quantities =  [v*om_quantities[0] for k, v in values.om_material_bom[om_matnrs[0]]["materials"].items()]
    # Proc quantities
    quantity_factor = random.randint(24, 48) if values.om_plants[plant]['high_value'] else random.randint(6, 18)
    quantities = [random.randint(12, 60)*quantity_factor for temp_quantity_index in range(len(matnrs))]
    has_qty_change = True if values.om_company_codes[company_code]['incorrect_qty_prbobability'] > 0.3 else False

    # PRICES and CONTRACTS
    #OM prices
    om_prices = [all_prices[om_matnrs[temp_matnr_index]] for temp_matnr_index in range(len(om_matnrs))]
    #Bom prices
    bom_prices = [all_prices[bom_matnrs[temp_matnr_index]] for temp_matnr_index in range(len(bom_matnrs))]
    #Proc prices
    prices = [all_prices[matnrs[temp_matnr_index]] for temp_matnr_index in range(len(matnrs))]
    has_contract_probabilities = [all_has_contract_probabilities[matnrs[temp_contract_index]] for temp_contract_index in range(len(matnrs))]
    item_has_contract = [True if random.random() < has_contract_probabilities[j] else False for j in range(len(has_contract_probabilities))]
    has_contract = True 
    for hc in item_has_contract:
        if hc == False:
            has_contract = False
            break
    
    # delivery status
    delivery_status = [] # early: -x | on_time: 0 | late: x
    for temp_delivery_status_index in range(len(matnrs)):
        extra_days = 0
        if lifnr in ('VND1','VND6','VND14') and random.random() > 0.2:
             extra_days = random.randint(2, 14) # late
        elif all_availabilities[matnrs[temp_delivery_status_index]] < 0.3: 
            extra_days = random.randint(2, 14) # late
        elif (all_availabilities[matnrs[temp_delivery_status_index]] >= 0.3) and (all_availabilities[matnrs[temp_delivery_status_index]] < 0.8):
            extra_days = 0 # on time
        else:
            extra_days = -random.randint(2, 5)
        delivery_status.append(extra_days)
        
    
    # requested dev date
    requested_dev_date = random.randint(7, 17)
    
    # free text
    is_free_text = True if random.random() < values.om_company_codes[company_code]['free_text_pr_probability'] else False
    free_text_materials = [all_matnr_names[matnr] for matnr in matnrs] # TODO change with material description

    # time distribution
    pr_req_years = [2023, 2024]
    pr_req_year_probability = np.array([0.5, 0.5])
    pr_req_year_probability = pr_req_year_probability / pr_req_year_probability.sum(axis=0, keepdims=True)
    pr_req_year = pr_req_years[np.random.choice(2, p=pr_req_year_probability)]
    if pr_req_year == 2023:
        pr_req_month_distribution = np.array([7, 5, 5, 4, 2, 6, 3, 5, 5, 6, 7, 10])
        pr_req_month_probability = pr_req_month_distribution / pr_req_month_distribution.sum(axis=0, keepdims=True)
        pr_req_month = np.random.choice(12, p=pr_req_month_probability) + 1 # because months should be in [1, 12] for datetime
    else:
        pr_req_month_distribution = np.array([7, 5, 5, 4, 2, 6, 3, 5, 7, 8])
        pr_req_month_probability = pr_req_month_distribution / pr_req_month_distribution.sum(axis=0, keepdims=True)
        pr_req_month = np.random.choice(10, p=pr_req_month_probability) + 1 # because months should be in [1, 12] for datetime



    params = {
        "om_matnrs" : om_matnrs,
        "bom_matnrs" : bom_matnrs,
        'matnrs': matnrs,
        
        'konnr': konnr,
        'kunnr': kunnr,
        'lifnr': lifnr,
        'plant': plant,
        
        'om_quantities':  om_quantities,
        "bom_quantities": bom_quantities,
        'quantities': quantities,
        
        'om_prices': om_prices,
        'bom_prices' : bom_prices,
        'prices': prices, # considered contract and PO prices
        
        'company_code': company_code,
        'purchasing_org': purchasing_org,
        'payment_term': payment_term,
        'requested_by': requested_by,
        'requested_dev_date' : requested_dev_date,
        'item_has_contract': item_has_contract,
        'has_contract': has_contract,
        'is_free_text': is_free_text,
        'free_text_materials': free_text_materials,

        'has_qty_change': has_qty_change,

        'has_payment_term_change': random.random() > values.proc_vendors[vendor_name]['payment_term_stsability'],
        'delivery_status': delivery_status,

        'new_payment_term': 'Z090',
        'new_vendor': new_lifnr,
        
        'pr_req_year': pr_req_year,
        'pr_req_month': pr_req_month,
        'automation_rate': automation_rate,
        'sales_doc_type': sales_doc_type,
        'distribution_channel': distribution_channel,
        'sales_org': sales_org,
        'sales_office': sales_office,
    }

    return params

In [ ]:
inventory_doc_tables = {
    'EBAN_json': {}, 
	'CDHDR_json': {},
	'CDPOS_json': {},
	'EKKO_json': {},
	'EKPO_json': {},
	'NAST_json': {},
	'MSEG_json': {},
	'EKBE_json': {},
	'RBKP_json': {},
	'RSEG_json': {},
	'EKET_json': {},
	'AFKO_json': {},
	'AFPO_json': {},
	'VBAK_json': {},
	'VBAP_json': {},
	'AUFK_json': {},
	'VBFA_json': {},
	'LIKP_json': {},
	'LIPS_json': {},
	'VBKD_json': {},
	'VBUK_json': {},
	'VBEP_json': {},
	'MBEW_json': {},
	'MBEWH_json': {},
}

#INVENTORY MANAGEMENT EVENTS LIST

for i in range(1000):
# for i in range(1_000):
	transition_prob = np.array([
	  # [0     1     2     3     4     5     6     7     8     9     10    11    12    13    14    15    16    17    18  ] 
		[0.00, 1.00, 0.00, 0.00, 0.00, 0.05], # 0 Create SalesOrder
		[0.00, 0.00, 1.00, 0.00, 0.00, 0.10], # 1 Create ProductionOrderHeader  includes Begin and Finish
		[0.00, 0.00, 0.00, 1.00, 0.00, 0.10], # 2 Create PurchaseOrderItem
		[0.00, 0.00, 0.00, 0.00, 1.00, 0.01], # 3 Post Goods Receipt
		[0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # 4 Post Goods Issue 
		[0.00, 0.00, 0.00, 0.00, 0.00, 1.00]  # 5 Terminate
	])
# NOT INCLUDED		[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # 7 Reverse Goods Issue
	
	params = get_params()
	latest_date = datetime(params['pr_req_year'], params['pr_req_month'], random.randint(1, 28)).date()
	latest_time = helpers.generate_random_time()
	inv = inventory_doc_data.Inventory(params=params, start_date=latest_date, index=i, vbeln = f'{str(uuid.uuid4())[-11:]}')
	preq_creation_date = latest_date
	so_created_date = latest_date
	reqested_delivery_date= latest_date + timedelta (days = params['requested_dev_date'])
	so_created_by = get_user_name(0.9*params['automation_rate'])
	pr_automation_rate = 0.2 if params['is_free_text'] else 0.7

	step = 0 # Create SO
	creation_time = helpers.generate_random_time()
	purchase_creation_date = latest_date

	while step != 5:
		if step == 0:
			inv.create_sales_order(
			reqested_delivery_date=reqested_delivery_date,
			shipping_condition=random.choice(list(values.shipping_conditions.keys())),
			erdat=latest_date,
			ernam=so_created_by,
			atime=latest_time
    		)
			latest_date += helpers.UPTO_WEEK()

		elif step == 1:
			inv.create_production_order_header(
				cpudt=latest_date,
				atime=latest_time,
				ernam = get_user_name(0.2)
			)
			latest_date += helpers.UPTO_WEEK()

		elif step == 2:
			inv.create_purchase_order(
				aedat=latest_date,
				ernam=get_user_name(0.8),
				utime=creation_time
			)
			latest_date += helpers.UPTO_WEEK()

		elif step == 3:
			inv.goods_receipt(
				cpudt=latest_date,
				usnam=get_user_name(0.7),
				atime=latest_time,
				udate=purchase_creation_date,
				utime=creation_time
			)
			latest_date += helpers.UPTO_WEEK()

		elif step == 4:
			inv.post_goods_issue(
                cpudt=latest_date,
                usnam=get_user_name(0.6*params['automation_rate']),
                atime=helpers.generate_random_time()         
            )
		
		
		# normalize transition matrix
		transition_prob = transition_prob / transition_prob.sum(axis=1, keepdims=True)
		step = np.random.choice(6, p=transition_prob[step])
	
	for k, v in inv.tables.items():
		for entry_key in list(v.keys()):
			inventory_doc_tables[k][entry_key] = inv.tables[k][entry_key]

In [ ]:
all_matnrs = MARA['MATNR'].unique()
all_plants = MARC['WERKS'].unique()

for p in all_plants:
    for m in all_matnrs:
        for y in range(2023,2025):
            if y == 2023:
                for mo in range(1,13):
                    inv.material_stock_history(plant=p, total_quantity=random.randint(24,67), matnr=m, price=all_prices[m], year=y, month=mo)
            else:
                for mo2 in range (1,11):
                    if mo2 == 10:
                        inv.latest_material_stock(plant=p, total_quantity=random.randint(24,67), matnr=m, price=all_prices[m], year=y, month=mo2) 
                    else:
                        inv.material_stock_history(plant=p, total_quantity=random.randint(24,67), matnr=m, price=all_prices[m], year=y, month=mo2)  

for k, v in inv.tables.items():
    for entry_key in list(v.keys()):
        inventory_doc_tables[k][entry_key] = inv.tables[k][entry_key]

In [ ]:
for table, rows in inventory_doc_tables.items():
    table_name = table.split('_')[0]
    # all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
    # all_cols_req = ctl.clean_columns(all_cols,table_name)
    all_cols_req = pd.DataFrame(columns= ctl.required_table_columns[table_name])
    df = pd.concat([all_cols_req, pd.DataFrame(rows.values())])
    df.to_csv(f'data/inv/ocpm/inventory_doc/{table_name}.csv', index=False)

In [ ]:
# print(inventory_doc_tables['MSEG_json'])
print(all_plants)
print(all_matnrs)

for p in all_plants:
    for m in all_matnrs: